In [252]:
site_id = "SF14136A"
address = '100 South Mathilda Avenue, Sunnyvale, California 94086'

In [253]:
from io import StringIO
from docx.shared import Cm
from docxtpl import DocxTemplate, InlineImage
import pandas as pd
from math import log10
import requests


In [254]:
def qb_dataframe(table_id):
    # Get fields
    url = "https://api.quickbase.com/v1/fields?tableId=" + table_id
    payload = {}
    headers = {
      'QB-Realm-Hostname': 'soteriarf',
      'Authorization': 'QB-USER-TOKEN b4pp4v_paqb_eyxwg9bec3v8pb63hjwjb9t36ii',
      'Content': 'application.json'
    }
    response = requests.request("GET", url, headers=headers, data = payload)
    df = pd.json_normalize(response.json())
    df = df[["label", "id", "fieldType", "mode"]]
    df = df.sort_values('id').reset_index(drop=True)
    df = df[df['fieldType'] != 'user']
    df = df[df['fieldType'] != 'file']
    label_list = list(df['label'])
    id_list = list(df['id'])
    #print(label_list, id_list)
    
    # Query the table
    headers = {
      'QB-Realm-Hostname': 'soteriarf.quickbase.com',
      'Authorization': 'QB-USER-TOKEN b4pp4v_paqb_eyxwg9bec3v8pb63hjwjb9t36ii',
      'Content-Type': 'application/json',
    }
    body = {
        "from": table_id,
        "select": id_list,
        "options": {
            "skip": 0,
            "top": 10000,
            #"compareWithAppLocalTime": false
        }} #, "where": "{6.EQ.'" + site_id + "'}"}
    r = requests.post(
        'https://api.quickbase.com/v1/records/query', 
        headers = headers, 
        json = body
    )
    df = pd.json_normalize(r.json()['data'])
    df.columns = df.columns.str.replace(".value", "")
    
    # Rename the dataframe with the correct fields names
    id_list = [str(i) for i in id_list] 
    for col in df.columns:
        df.rename(columns={col:label_list[id_list.index((col))]}, inplace=True)
    return df

In [279]:
# Function to convert from W to dBm
def W2dBm(W):
    return 10*log10(W*1000)

# Function to convert from dBm to mW
def dBm2W(dBm):
    return 10**((dBm)/10) / 1000

In [282]:
w2dbm(1862)

62.69979676645324

In [284]:
gain_dbd = 15.7

In [286]:
dBm2W(w2dbm(661) - 12.4)

38.03657985798617

In [256]:
# Load Transmitters Table
df = qb_dataframe('bqzaae4x9')

# Query on the site
df = df[df['Antenna Location - SITE_ID'] == site_id]
df.head()

,Date Created,Input Power (W),Date Modified,Pattern Code,Pattern Code - Aperture (Feet),Pattern Code - Gain (dBd),Pattern Code - Horizontal BW,Pattern Code - Vertical BW,Record ID#,Pattern Code - Manufacturer,...,Antenna Location - y_roofview,Antenna Location - SITE_ID - Rooftop AGL,Antenna Location - SITE_ID - Roofview AGL,Antenna Location - ANTENNA_ID,Antenna Location - SECTOR,Loss (dB),Antenna Location - RAD_CENTER_FT,Technology,Port,Frequency MHz
0,2020-12-09T00:02:02Z,100.0,2020-12-16T17:33:09Z,AIR6449_2500,2.759183,22.65,60.0,7.5,322,Ericsson,...,106.583438,92.0,92.0,SF14136A_TMO_A_1,A,NaN,96,LTE,P01,2500.0
1,2020-12-09T00:08:46Z,80.0,2020-12-14T17:45:10Z,AIR32_2100,6.466667,15.35,59.0,5.2,326,Ericsson,...,106.353574,92.0,92.0,SF14136A_TMO_A_2,A,NaN,96,,,2100.0
2,2020-12-09T00:10:24Z,160.0,2020-12-14T17:45:10Z,AIR32_1900,6.466667,15.35,60.0,5.6,329,Ericsson,...,106.353574,92.0,92.0,SF14136A_TMO_A_2,A,NaN,96,,,1900.0
3,2020-12-09T00:12:19Z,80.0,2020-12-14T17:45:10Z,APXVAARR18_43-U-NA20_700,6.000000,12.65,63.0,13.0,332,RFS,...,106.118685,92.0,92.0,SF14136A_TMO_A_3,A,NaN,96,,,700.0
4,2020-12-09T00:12:35Z,160.0,2020-12-14T17:45:10Z,APXVAARR18_43-U-NA20_600,6.000000,12.25,66.0,14.0,333,RFS,...,106.118685,92.0,92.0,SF14136A_TMO_A_3,A,NaN,96,,,600.0


In [257]:
list(df)

['Date Created',
 'Input Power (W)',
 'Date Modified',
 'Pattern Code',
 'Pattern Code - Aperture (Feet)',
 'Pattern Code - Gain (dBd)',
 'Pattern Code - Horizontal BW',
 'Pattern Code - Vertical BW',
 'Record ID#',
 'Pattern Code - Manufacturer',
 'z_roofview',
 'sort_key',
 'Related Antenna Location',
 'Antenna Location - SITE_ID',
 'Antenna Location - ANTENNA_MODEL',
 'Antenna Location - AZIMUTH',
 'Antenna Location - SITE_ID - Scale',
 'Antenna Location - x_roofview',
 'Antenna Location - y_roofview',
 'Antenna Location - SITE_ID - Rooftop AGL',
 'Antenna Location - SITE_ID - Roofview AGL',
 'Antenna Location - ANTENNA_ID',
 'Antenna Location - SECTOR',
 'Loss (dB)',
 'Antenna Location - RAD_CENTER_FT',
 'Technology',
 'Port',
 'Frequency MHz']

In [258]:
df = df.rename(columns={
    'Antenna Location - SECTOR': 'sector',
    'Port': 'port',
    'Technology': 'technology',
    'Frequency MHz': 'freq',
    'Pattern Code - Manufacturer': 'manufacturer',
    'Antenna Location - ANTENNA_MODEL': 'model',
    'Antenna Location - AZIMUTH': 'azimuth',
    'Antenna Location - RAD_CENTER_FT': 'agl',
    'Input Power (W)': 'power',
    'Pattern Code - Gain (dBd)': 'gain'
    })
df.head()

,Date Created,power,Date Modified,Pattern Code,Pattern Code - Aperture (Feet),gain,Pattern Code - Horizontal BW,Pattern Code - Vertical BW,Record ID#,manufacturer,...,Antenna Location - y_roofview,Antenna Location - SITE_ID - Rooftop AGL,Antenna Location - SITE_ID - Roofview AGL,Antenna Location - ANTENNA_ID,sector,Loss (dB),agl,technology,port,freq
0,2020-12-09T00:02:02Z,100.0,2020-12-16T17:33:09Z,AIR6449_2500,2.759183,22.65,60.0,7.5,322,Ericsson,...,106.583438,92.0,92.0,SF14136A_TMO_A_1,A,NaN,96,LTE,P01,2500.0
1,2020-12-09T00:08:46Z,80.0,2020-12-14T17:45:10Z,AIR32_2100,6.466667,15.35,59.0,5.2,326,Ericsson,...,106.353574,92.0,92.0,SF14136A_TMO_A_2,A,NaN,96,,,2100.0
2,2020-12-09T00:10:24Z,160.0,2020-12-14T17:45:10Z,AIR32_1900,6.466667,15.35,60.0,5.6,329,Ericsson,...,106.353574,92.0,92.0,SF14136A_TMO_A_2,A,NaN,96,,,1900.0
3,2020-12-09T00:12:19Z,80.0,2020-12-14T17:45:10Z,APXVAARR18_43-U-NA20_700,6.000000,12.65,63.0,13.0,332,RFS,...,106.118685,92.0,92.0,SF14136A_TMO_A_3,A,NaN,96,,,700.0
4,2020-12-09T00:12:35Z,160.0,2020-12-14T17:45:10Z,APXVAARR18_43-U-NA20_600,6.000000,12.25,66.0,14.0,333,RFS,...,106.118685,92.0,92.0,SF14136A_TMO_A_3,A,NaN,96,,,600.0


In [259]:
df = df[['Related Antenna Location', 'sector', 'port', 'technology', 'freq', 'manufacturer', 'model', 'azimuth', 'agl', 'power', 'gain']]
df.head()

,Related Antenna Location,sector,port,technology,freq,manufacturer,model,azimuth,agl,power,gain
0,SF14136A_TMO_A_1,A,P01,LTE,2500.0,Ericsson,AIR6449,10.0,96,100.0,22.65
1,SF14136A_TMO_A_2,A,,,2100.0,Ericsson,AIR32,10.0,96,80.0,15.35
2,SF14136A_TMO_A_2,A,,,1900.0,Ericsson,AIR32,10.0,96,160.0,15.35
3,SF14136A_TMO_A_3,A,,,700.0,RFS,APXVAARR18_43-U-NA20,10.0,96,80.0,12.65
4,SF14136A_TMO_A_3,A,,,600.0,RFS,APXVAARR18_43-U-NA20,10.0,96,160.0,12.25


In [260]:
df['erp_dbm'] = ''
df['erp'] = ''
for x in range(len(df)):
    df['erp_dbm'].iloc[x] = round(mW2dBm(df['power'].iloc[x]*1000) + df['gain'].iloc[x], 1)
    df['erp'].iloc[x] = int(dBm2mW(df['erp_dbm'].iloc[x]) / 1000)
df = df.drop('erp_dbm', 1)
df['freq'] = df['freq'].astype(int)
df['azimuth'] = df['azimuth'].astype(int)
df['power'] = df['power'].astype(int)
df = df.round({'gain': 1})
df.sort_values(['Related Antenna Location', 'port', 'freq'], ascending=[True, True, True], inplace=True)
df.head()

,Related Antenna Location,sector,port,technology,freq,manufacturer,model,azimuth,agl,power,gain,erp
0,SF14136A_TMO_A_1,A,P01,LTE,2500,Ericsson,AIR6449,10,96,100,22.6,18197
18,SF14136A_TMO_A_1,A,P01,N,2500,Ericsson,AIR6449,10,96,40,22.6,7413
19,SF14136A_TMO_A_1,A,P02,LTE,2500,Ericsson,AIR6449,10,96,40,22.6,7413
20,SF14136A_TMO_A_1,A,P02,N,2500,Ericsson,AIR6449,10,96,100,22.6,18197
2,SF14136A_TMO_A_2,A,,,1900,Ericsson,AIR32,10,96,160,15.4,5495


In [261]:
df.iloc[0].to_json(r'json_out.json')
# {"sector":"A","technology":"LTE","freq":2500.0,"manufacturer":"Ericsson","model":"AIR6449","azimuth":10.0,"agl":"96","power":280.0,"gain":22.65,"erp_dbm":77.1,"erp":51286}

In [262]:
row_contents = []
for x in range(len(df)):
    print(df.iloc[x].to_json())
    row_contents.append(eval(df.iloc[x].to_json()))
row_contents

{"Related Antenna Location":"SF14136A_TMO_A_1","sector":"A","port":"P01","technology":"LTE","freq":2500,"manufacturer":"Ericsson","model":"AIR6449","azimuth":10,"agl":"96","power":100,"gain":22.6,"erp":18197}
{"Related Antenna Location":"SF14136A_TMO_A_1","sector":"A","port":"P01","technology":"N","freq":2500,"manufacturer":"Ericsson","model":"AIR6449","azimuth":10,"agl":"96","power":40,"gain":22.6,"erp":7413}
{"Related Antenna Location":"SF14136A_TMO_A_1","sector":"A","port":"P02","technology":"LTE","freq":2500,"manufacturer":"Ericsson","model":"AIR6449","azimuth":10,"agl":"96","power":40,"gain":22.6,"erp":7413}
{"Related Antenna Location":"SF14136A_TMO_A_1","sector":"A","port":"P02","technology":"N","freq":2500,"manufacturer":"Ericsson","model":"AIR6449","azimuth":10,"agl":"96","power":100,"gain":22.6,"erp":18197}
{"Related Antenna Location":"SF14136A_TMO_A_2","sector":"A","port":"","technology":"","freq":1900,"manufacturer":"Ericsson","model":"AIR32","azimuth":10,"agl":"96","power":

[{'Related Antenna Location': 'SF14136A_TMO_A_1',
  'sector': 'A',
  'port': 'P01',
  'technology': 'LTE',
  'freq': 2500,
  'manufacturer': 'Ericsson',
  'model': 'AIR6449',
  'azimuth': 10,
  'agl': '96',
  'power': 100,
  'gain': 22.6,
  'erp': 18197},
 {'Related Antenna Location': 'SF14136A_TMO_A_1',
  'sector': 'A',
  'port': 'P01',
  'technology': 'N',
  'freq': 2500,
  'manufacturer': 'Ericsson',
  'model': 'AIR6449',
  'azimuth': 10,
  'agl': '96',
  'power': 40,
  'gain': 22.6,
  'erp': 7413},
 {'Related Antenna Location': 'SF14136A_TMO_A_1',
  'sector': 'A',
  'port': 'P02',
  'technology': 'LTE',
  'freq': 2500,
  'manufacturer': 'Ericsson',
  'model': 'AIR6449',
  'azimuth': 10,
  'agl': '96',
  'power': 40,
  'gain': 22.6,
  'erp': 7413},
 {'Related Antenna Location': 'SF14136A_TMO_A_1',
  'sector': 'A',
  'port': 'P02',
  'technology': 'N',
  'freq': 2500,
  'manufacturer': 'Ericsson',
  'model': 'AIR6449',
  'azimuth': 10,
  'agl': '96',
  'power': 100,
  'gain': 22.6,
 

In [263]:
context = {
    'site_id': site_id,
    'address': address,
    'row_contents': row_contents
    }

In [264]:
doc = DocxTemplate('AntennaInventoryTpl.docx')
doc.render(context)
doc.save('test.docx')

In [ ]:
dBm2W(w2dbm(661) - 12.4)